<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/agents_and_routing/examples_agents_with_critique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install ollama langchain_community --quiet

host="localhost:11434"
modelid="llama3.2"
emb_modelid="mxbai-embed-large"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {modelid}")
get_ipython().system_raw(f"ollama pull {emb_modelid}")


In [10]:
!pip install openai --quiet

In [11]:
get_ipython().system_raw(f"ollama pull {modelid}")

In [12]:
from openai import OpenAI


llm = OpenAI(
        base_url=f"http://{host}/v1",
        api_key="ollama",  # required, but unused
    )




In [13]:
def call(system_prompt: str, message: str) -> str:
  completion = llm.chat.completions.create(
      model=modelid,
      messages=[
          {"role": "system", "content": system_prompt},
          {
              "role": "user",
              "content": message,
          },
      ],
  )

  return completion.choices[0].message.content

In [14]:
def marketing_campaign_agent(product : str):
  """
     <PRODUCT>{product}</PRODUCT>

     Your task is to create an compelling marketing campaign for the product provided between the <product> tag.
  """
  return call ("your are a marketing campaign export.", marketing_campaign_agent.__doc__.format(product=product) )

In [18]:
def ready_validation_agent(marketing_text : str) -> bool:
  """

    <CONTENT>{text}</CONTENT>
    I have asked you to create a marketing text for a product  earlier. You can find the text between the <CONTENT> tags.

    Is this the final version of the marketing text and ready to publish or can we improve it?

     Your task is to answer with
     - TRUE if this is the final version of the marketing text and ready to publish.
     - FALSE if this is not the final version of the marketing text and we need to improve it.

     Only answer with TRUE or FALSE. add nothing else to your response.

    """
  result = call ("your are a marketing campaign export.", ready_validation_agent.__doc__.format(text=marketing_text) )
  print (result)
  return result.strip().lower() == "true"

In [31]:
def refinements_agent(product : str, marketing_text : str) -> str:
  """
  <PRODUCT>{product}</PRODUCT>

  <CONTENT>{text}</CONTENT>

  I have asked you to create a marketing text for a product earlier. You can find the text between the <CONTENT> tags.

  This marketing text is not final and not yet ready for publishing.

  Your task is to refine the marketing text provided betwen the <CONTENT> tag with the follwing refinements. Don't use any other sources.

  ### OUTPUT
  Please output a  list of improvements you have found.

  """
  return call ("your are a marketing campaign export.", refinements_agent.__doc__.format(product=product, text=marketing_text) )

In [36]:
def refine_agent(product : str, marketing_text : str, refinements : str ) -> str:
  """
  <PRODUCT>{product}</PRODUCT>

  <CONTENT>{text}</CONTENT>

  <REFINE>{refinements}</REFINE>

  I have asked you to create a marketing campaign for a product earlier. You can find the markeging campaign between the <CONTENT> tags.

  Your task is to add the refinements provided between <REFINE> tags to that marketing campaign .



  """
  return call ("your are a marketing campaign export.", refinements_agent.__doc__.format(product=product, text=marketing_text, refinements=refinements) )

In [37]:
product = "iPhone 15"

marketing_text = marketing_campaign_agent(product)
is_finished = ready_validation_agent(marketing_text)
refinements = refinements_agent(product, marketing_text)
next_text_iteration = refine_agent(product, marketing_text, refinements)


FALSE


In [40]:
print (next_text_iteration)


<PRODUCT>iPhone 15</PRODUCT>

<CONTENT>Here's a revised and refined comprehensive marketing campaign for the iPhone 15:

**Campaign Name:** "Unlock the Future: Experience the Power of iPhone 15"

**Objective:** To create an immersive experience that showcases the iPhone 15 as the ultimate smartphone for the modern user, emphasizing its innovative features, speed, and style.

**Target Audience:**

* Demographics: Tech-savvy individuals aged 18-45
* Interests: Mobile gaming, music production, photography, and social media influencers
* Psychographics: People who value innovation, speed, and style

**Marketing Strategies:**

1. **Teaser Campaign**
	* Release a series of captivating ads showcasing the iPhone 15's cutting-edge features (e.g., advanced camera system, AI-powered performance)
	* Utilize Instagram Reels, TikTok, and YouTube Shorts to create anticipation
2. **Influencer Partnerships**
	* Collaborate with top gaming influencers, photographers, and social media personalities to sh

In [35]:
print (marketing_text)
print ("###### critique")
print (is_finished)
print (refinements)
print ("###### next text iteration")
print (next_text_iteration)

Here's a comprehensive marketing campaign for the iPhone 15:

**Campaign Name:** "Evolve with iPhone 15"

**Objective:** To position iPhone 15 as the ultimate mobile experience that empowers users to evolve their lives, work, and play.

**Target Audience:**

* Young professionals (18-35 years)
* Tech-savvy individuals
* Creatives and entrepreneurs

**Key Messages:**

1. **Innovation Beyond Limits**: Showcase the cutting-edge features of iPhone 15, including improved cameras, advanced processing power, and longer battery life.
2. **Unlock Your Potential**: Highlight how iPhone 15's user-friendly interface and AI-powered tools empower users to unleash their creativity, productivity, and connectivity.
3. **Evolve with Every Moment**: Emphasize the phone's ability to capture life's precious moments, connect with loved ones across distances, and provide a seamless experience for work, play, and everyday living.

**Marketing Strategies:**

1. **Video Campaigns**:
	* "Unlock Your Potential" (